In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text") 

from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

import bs4
from langchain_community.document_loaders import WebBaseLoader

#bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

# loader = WebBaseLoader(
#     web_paths=("https://vinwonders.com/vi/wonderpedia/news/gioi-thieu-ve-ha-noi/",),
#     bs_kwargs={"parse_only": bs4_strainer},
# )

# #1.  Thêm User-Agent header:
# loader = WebBaseLoader(
#     web_paths=("https://vinwonders.com/vi/wonderpedia/news/gioi-thieu-ve-ha-noi/",),
#     bs_kwargs={"parse_only": bs4_strainer},
#     header_template={"User-Agent": "Mozilla/5.0"}
# )

# #2. 2. Kiểm tra class names thực tế - thử load toàn bộ trang trước:
# loader = WebBaseLoader("https://vinwonders.com/vi/wonderpedia/news/gioi-thieu-ve-ha-noi/")
# docs = loader.load()
# print(docs[0].page_content[:500])
# # => Vấn đề: Website yêu cầu JavaScript và cookies - 
# # WebBaseLoader không thể xử lý vì nó chỉ gửi HTTP request đơn giản, không chạy JavaScript.
# # Giải pháp: Dùng các loader hỗ trợ JavaScript:
# # 1. Dùng PlaywrightURLLoader (khuyến nghị):

# loader = WebBaseLoader(
#     web_paths=("https://vi.wikipedia.org/wiki/H%C3%A0_N%E1%BB%99i",)
# )
bs4_strainer = bs4.SoupStrainer(id="mw-content-text")
loader = WebBaseLoader(
    web_paths=("https://vi.wikipedia.org/wiki/H%C3%A0_N%E1%BB%99i",),
    bs_kwargs={"parse_only": bs4_strainer}
)

docs = loader.load()
assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}") # Load được bao nhiêu kí tự?
#print(docs[0].page_content[:500]) # [:500]: Slicing - chỉ lấy 500 ký tự đầu tiên của chuỗi văn bản
#print(docs[0].page_content) # lấy ra toàn bộ nội dung

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs) # chia được bao nhiêu document
print(f"Split blog post into {len(all_splits)} sub-documents.")


# Add documents vào vector store: vector_store.add_documents(all_splits)
# - Lưu các chunks đã split vào vector store
document_ids = vector_store.add_documents(documents=all_splits)
#print(document_ids[:3])

# Retrieval (Truy xuất) - Test similarity search

from langchain.tools import tool
@tool(response_format="content_and_artifact")

def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs 
# Trả về 2 giá trị: văn bản (cho model) và documents gốc (cho ứng dụng)

# Tạo Agentic RAG:
# Import thư viện ChatOllama để sử dụng mô hình Ollama
from langchain_ollama import ChatOllama
# Khởi tạo mô hình chat với tên"gpt-oss" và temperature=0 (để kết quả ổn định, không ngẫu nhiên)
model = ChatOllama(model="gpt-oss", temperature=0)
# Import hàm create_agent để tạo agent
from langchain.agents import create_agent
# Định nghĩa danh sách các công cụ (tools) mà agent có thể sử dụng
tools = [retrieve_context] 

prompt = """Bạn chỉ được phép trả lời dựa trên CONTEXT bên dưới.
Không được sử dụng kiến thức bên ngoài.
Nếu câu hỏi liên quan đến thông tin không có trong CONTEXT,
hãy trả lời đúng một câu: Trong tài liệu không có thông tin này.""""

agent = create_agent(model, tools, system_prompt = prompt)
query = ("Hãy tóm tắt nội dung chính")
for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()


 ········


C:\Users\cao_ngoc_dung\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 160573
Split blog post into 248 sub-documents.
================================ Human Message =================================

Hãy tóm tắt nội dung chính
================================== Ai Message ==================================

Xin lỗi, hiện tại tôi không có đủ thông tin để tóm tắt nội dung chính. Nếu bạn cung cấp nội dung hoặc chủ đề cụ thể, mình sẽ giúp bạn tóm tắt ngay!
